In [87]:
import time
import statistics
import serial
import serial.tools.list_ports


In [48]:
com_ports=serial.tools.list_ports.comports()
for p in com_ports:
    print(p.device)

/dev/ttyS31
/dev/ttyS30
/dev/ttyS29
/dev/ttyS28
/dev/ttyS27
/dev/ttyS26
/dev/ttyS25
/dev/ttyS24
/dev/ttyS23
/dev/ttyS22
/dev/ttyS21
/dev/ttyS20
/dev/ttyS19
/dev/ttyS18
/dev/ttyS17
/dev/ttyS16
/dev/ttyS15
/dev/ttyS14
/dev/ttyS13
/dev/ttyS12
/dev/ttyS11
/dev/ttyS10
/dev/ttyS9
/dev/ttyS8
/dev/ttyS7
/dev/ttyS6
/dev/ttyS5
/dev/ttyS4
/dev/ttyS3
/dev/ttyS2
/dev/ttyS1
/dev/ttyS0
/dev/ttyACM0


In [49]:
arduino = serial.Serial('/dev/ttyACM0', baudrate=115200, timeout=1)

In [50]:
def handshake_arduino(arduino, sleep_time=1):
    time.sleep(sleep_time);
    timeout = arduino.timeout
    arduino.timeout = 2
    arduino.write('h'.encode())
    while (arduino.in_waiting < 0):
        pass
    response= arduino.read_until()
    print(response.decode().strip())
    arduino.timeout = timeout

In [51]:
handshake_arduino(arduino,3)

R


In [69]:
def parse_raw_line(raw_line):
    raw_line = raw_line.decode()
    if raw_line[-1] != "\n":
        raise ValueError(
            "Il messaggio inviato da Arduino deve terminare con il carattere nuova linea."
        )
    
    broadband,ir= raw_line.rstrip().split("|")

    return int(broadband),int(ir)

In [79]:
def get_light_reading(n_samples,integration_time):
    #n_samples: average over n_samples
    #sensor integration time, defaults to 402ms
    broadband_readings=[]
    ir_readings=[]
    
    for reading in range(n_samples):
            # invio il comando per la lettura del sensore TSL2561
            arduino.write('r'.encode())
            while arduino.in_waiting < 0:
                pass
            # Leggo i dati inviati tramite seriale a seguito della richiesta di lettura
            raw = arduino.read_until()
    
            broadband,ir=parse_raw_line(raw)
            broadband_readings.append(broadband)
            ir_readings.append(ir)
            time.sleep(1.5*integration_time/1000)
    return int(statistics.mean(broadband_readings)),int(statistics.mean(ir_readings))

    

Test the reading function

In [80]:
get_light_reading(15,402)

(14, 2)

Lux convertion function as per TSL2561 datasheet

In [99]:
TSL2561_CLIPPING_402MS:int=65000
TSL2561_LUX_CHSCALE:int=10
TSL2561_LUX_RATIOSCALE:int =9

def calculateLux(broadband:int,ir:int):
    clipThreshold = TSL2561_CLIPPING_402MS
    chScale = (1 << TSL2561_LUX_CHSCALE)
    channel0:int = (broadband * chScale) >> TSL2561_LUX_CHSCALE
    channel1:int = (ir * chScale) >> TSL2561_LUX_CHSCALE
    ratio:float=0
    lux:float=0
    if (channel0 != 0):
        ratio =channel1/channel0
        
    if ratio>0 and ratio <=0.5:
        lux=0.0304*channel0-0.062*channel0*pow(ratio,1.4)
    elif ratio>0.5 and ratio<=0.61:
        lux=0.0224*channel0-0.031*channel1
    elif ratio>0.61 and ratio <=0.80:
        lux=0.0128*channel0-0.0153*channel1
    elif ratio>0.80 and ratio <=1.30:
        lux=0.00146*channel0-0.00112*channel1
    elif ratio>1.30 and ratio <=3.00:
        lux=0.0
    
    return round(lux,1)

In [100]:
data=get_light_reading(10,402)
calculateLux(data[0],data[1])

0